In [2208]:
import pandas as pd
import numpy as np

import os

import warnings
warnings.filterwarnings('ignore')

## Ввести абсолютный путь к файлам

In [2209]:
# path = "C:\\Users\\mi\\jupyterProjects\\tom's work"
path = "/Users/valeriali/JupyterProjects/tom's work"

In [2210]:
os.chdir(path)

In [2211]:
os.getcwd()

"/Users/valeriali/JupyterProjects/tom's work"

In [2212]:
#настройки для отображения всех колонок
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

### Считаваем файлы req-implements

In [2213]:
for i in os.listdir("req_implement"):
    print(i)

.DS_Store
req-implement.txt


In [2214]:
columns_for_req = ['№ зкз.Покуп','Дата разнарядки с учетом приоритета','КССС мат','Номенклатура материала', 'Заявлено Покупателем, тн', 
                   'Не исполнено по продукту, тн', 'Отгружено, тн', 'Покупатель', 'Вид транспортного средства', 
                   'СтКМ', 'Комментарий', 'ВремяСтКМ', 'ДатСтКМ', 'Время', 'ПланКОНдата', 'ПланНАЧдата', 'Инкотермс', 
                   'Район сбыта', 'Договор', 'ДатаСоздан', 'ДатаИзм', 'Рынок', 'Пункт отгрузки', 'Предписания по таре', 
                   'Период', 'Инкотермс 2', 'Регион ГП', 'Регион Грузополуч.', 'Маршрут', 'Пункт разгрузки', 'Тип дилера', 
                   'Исполнено факт.отгрузками', 'Фирменная / не фирменная проду', 'имя Грузополучателя', 'КССС Пок', 
                   'Условие отгрузки', 'СтКМ продукта', 'Стоимость', 'Валюта']

In [2215]:
orders = pd.read_csv('req_implement/req-implement.txt', encoding='ISO-8859-5', 
                     sep='\t',header=None, parse_dates=[1], dayfirst=True,
                     names=columns_for_req
                    )


In [2216]:
orders['Исполнено факт.отгрузками'] = orders['Исполнено факт.отгрузками'].str.replace(',','.').astype(float)
orders['Заявлено Покупателем, тн'] = orders['Заявлено Покупателем, тн'].str.replace(',','.').astype(float)
orders['Не исполнено по продукту, тн'] = orders['Не исполнено по продукту, тн'].str.replace(',','.').astype(float)
orders['Отгружено, тн'] = orders['Отгружено, тн'].str.replace(',','.').astype(float)

In [2217]:
orders.loc[11937]

№ зкз.Покуп                                               5493022
Дата разнарядки с учетом приоритета           2022-09-13 00:00:00
КССС мат                                                  3148675
Номенклатура материала                 Масло L GEN ARM 5W40 нк.4л
Заявлено Покупателем, тн                                  1222.56
                                                  ...            
КССС Пок                                                   151768
Условие отгрузки                                    Автотранспорт
СтКМ продукта                                                   A
Стоимость                                               787973,08
Валюта                                                        RUB
Name: 11937, Length: 39, dtype: object

In [2218]:
orders['ДатСтКМ'] = pd.to_datetime(orders['ДатСтКМ'])
orders['ДатаИзм'] = pd.to_datetime(orders['ДатаИзм'])
orders['ДатаСоздан'] = pd.to_datetime(orders['ДатаСоздан'])
orders['ПланКОНдата'] = pd.to_datetime(orders['ПланКОНдата'])
orders['ПланНАЧдата'] = pd.to_datetime(orders['ПланНАЧдата'])

In [2219]:
# заменяем строки в таблице
orders.loc[(orders['Пункт отгрузки'] == 'ООО "ЛУКОЙЛ-Торжок"'), 'Пункт отгрузки'] = 'ООО "Тебойл Рус" Торжок'

In [2220]:
orders['Пункт отгрузки'].unique()

array(['Пермнефтеоргсинтез ООО', 'Волгоградский НПЗ', 'Тюменский Филиал',
       'ООО "Тебойл Рус" Торжок', 'ООО "Тебойл Рус" Москва',
       'ООО "Тебойл Рус" С.-Петербург'], dtype=object)

In [2221]:
#удалим строки где в столбце номер заказа есть строки и приведем все к типу инт
orders['№ зкз.Покуп'] = pd.to_numeric(orders['№ зкз.Покуп'], errors='coerce')
orders.dropna(inplace=True, axis=0,subset=['№ зкз.Покуп'])
orders.reset_index(drop=True, inplace=True)
orders['№ зкз.Покуп'] = orders['№ зкз.Покуп'].astype(int)

### апдейт

In [2222]:
update = pd.read_excel('spravochnik.xlsx', sheet_name='апдейт')

In [2223]:
update

,№ зкз.Покуп,Дат.зкз.Пок_y
0,5488678,2022-09-24
1,5488679,2022-09-24
2,5490310,2022-09-24
3,5488684,2022-09-24
4,5488683,2022-09-24
...,...,...
70,5494558,2022-09-24
71,5494591,2022-09-24
72,5494589,2022-09-24
73,5494557,2022-09-24


In [2224]:
update_dates = pd.merge(orders, update, on=['№ зкз.Покуп'], how='left', suffixes=('', '_y'))



In [2225]:
update_dates = update_dates[update_dates['Дат.зкз.Пок_y'].notna()]



In [2226]:
orders.loc[update_dates.index, 'Дата разнарядки с учетом приоритета'] = update_dates['Дат.зкз.Пок_y']

In [2227]:
orders.sort_values(by=['Дата разнарядки с учетом приоритета','№ зкз.Покуп'], ascending=True, inplace=True)

### Считываем файл приоритеты и конкатенируем сверху

In [2228]:
prioritets = pd.read_excel('spravochnik.xlsx', sheet_name='приоритеты', 
                           converters={'ВремяСтКМ': lambda x: str(pd.Timedelta(x, 'd').round('s')).split(' ')[2],
                                      'Время': lambda x: str(pd.Timedelta(x, 'd').round('s')).split(' ')[2]})


In [2229]:
prioritets

,ключ,номер зкз,кссс,№ зкз.Покуп,Дата разнарядки с учетом приоритета,КССС мат,Номенклатура материала,Заявлено Покупателем,Не исполнено заявки,Исполнено опер.отгрузками,имя Покупателя,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,ИнкТм,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Канал сбыта,Пункт отгрузки,Предписания по таре,Период,Инкот. 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта
0,54862871599892,5486287,1599892,5486287,2022-09-05,1599892,"Масло L GENESIS SPEC A5/В5 5W30;б.216,5л",7480.00,0.00,7480.00,"АО ""Авилон АГ""",Авто для фасовки,C,NaN,11:11:29,44812,08:59:01,2022-09-30,2022-09-07,DDP,Оптовые поставки,20L0408,44809,44809,Внутренний рынок,Пермнефтеоргсинтез ООО,крупная фасовка,202209,Москва,77,г. Москва,Автотранспорт(Пермь->Москва),"МОСКОВСКАЯ ОБЛАСТЬ, Г. КОТЕЛЬНИКИ, КОММЕРЧЕСК...",NaN,7480.0,Фирм.не аналог имп/з,"АО ""Авилон АГ""",192281,Автотранспорт,A
1,5486138227349,5486138,227349,5486138,2022-09-01,227349,"Масло Л АВАНГАРД УЛЬТРА,10W30; б.216,5л",900.00,0.00,900.00,"ООО ""ЛУКОЙЛ-Пермнефтеоргсинтез""",Авто для фасовки,NaN,"Недогруз Август 2022 г., разнярядка № 0005485...",00:00:00,0,13:45:31,2022-09-30,2022-09-04,DDU,внутрикорп. Транзит,13L0555,44806,44810,Внутренний рынок,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Тюмень,77,г. Москва,Автотранспорт(Пермь->Тюмень),"П. БОГАНДИНСКИЙ, УЛ. НЕФТЯННИКОВ, 9",NaN,900.0,Фирм.не аналог имп/з,"Филиал ООО ""ЛЛК-Интернешнл"" в Тюмени",198349,Автотранспорт,A
2,54861253435821,5486125,3435821,5486125,2022-09-02,3435821,Масло Л АВ ПРОФ М5 10W40 б.175кг,0.00,0.00,0.00,"ООО ""Виланд""",Авто для фасовки,B,NaN,12:54:05,44810,12:38:04,2022-09-30,2022-09-04,CPT,Белоруссия,19L0442,44806,44817,Экспорт,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Минск,0,Нет данных,Автотранспорт(Пермь->г. Минск),"ТЖ:МИНСКАЯ ОБЛ.,ЩОМЫСЛИЦКИЙ С/С,17-Й КМ. А/Д ...",NaN,0.0,NaN,"ООО ""Виланд""",184884,Автотранспорт,E
3,5492131189509,5492131,189509,5492131,2022-09-07,189509,"Масло Л ЛЮКС ТУРБО ДИЗЕЛЬ 10W40 б.216,5л",720.00,0.00,720.00,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,44811,11:06:27,2022-09-30,2022-09-09,CPT,Белоруссия,19L0014,44811,44811,Экспорт,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Минск,7,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,720.0,Фирм. аналог имп/зам,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A
4,549213114913,5492131,14913,5492131,2022-09-07,14913,"Масло Л СУПЕР, 10W40, SG/CD; б.216,5л",5760.00,0.00,5760.00,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,44811,11:06:27,2022-09-30,2022-09-09,CPT,Белоруссия,19L0014,44811,44811,Экспорт,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Минск,7,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,5760.0,Фирм.не аналог имп/з,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,54920323451552,5492032,3451552,5492032,2022-08-10,3451552,Масло Omala S2 GX 320 в.20л,686.28,0.00,686.28,"ООО ""Промсервис-Центр""",Авто для фасовки,A,"Недогруз Август 2022 г., разнярядка № 0005482644",16:31:50,44790,07:42:30,2022-09-30,2022-09-09,DDP,Оптовые поставки,22L0857,44811,44811,Внутренний рынок,"ООО ""Тебойл Рус"" Торжок",средняя и мелкая фc.,202209,Москва,77,г. Москва,Автотранспорт (Торжок ->Москва),"РФ, 108851, МОСКВА Г, СТАРОСЫРОВО Д, СИМФЕРОП...",NaN,0.0,NaN,"ООО ""Промсервис-Центр""",153370,Автотранспорт,A
669,54920323451554,5492032,3451554,5492032,2022-08-10,3451554,Масло Omala S2 GX 220 в.20л,629.30,179.80,449.50,"ООО ""Промсервис-Центр""",Авто для фасовки,A,"Недогруз Август 2022 г., разнярядка № 0005482644",16:31:50,44790,07:42:30,2022-09-30,2022-09-09,DDP,Оптов

In [2230]:
prioritets['ДатСтКМ'] = pd.to_datetime(prioritets['ДатСтКМ'], unit='d', origin='1899-12-30')
prioritets['ДатаСоздан'] = pd.to_datetime(prioritets['ДатаСоздан'], unit='d', origin='1899-12-30')
prioritets['ДатаИзм'] = pd.to_datetime(prioritets['ДатаИзм'], unit='d', origin='1899-12-30')


In [2231]:
prioritets = prioritets.drop(columns=prioritets.iloc[:, range(3)]) 

In [2232]:
prioritets.rename(columns = {'Не исполнено заявки': 'Не исполнено по продукту, тн', 'Исполнено опер.отгрузками':'Отгружено, тн',
                            'Заявлено Покупателем':'Заявлено Покупателем, тн', 'имя Покупателя': 'Покупатель',
                            'ИнкТм':'Инкотермс','Канал сбыта':'Рынок','Инкот. 2':'Инкотермс 2'
                            }, inplace=True)

## конкатенируем сверху таблицу приоритетов

In [2233]:
orders = pd.concat([prioritets, orders], ignore_index=True)

In [2234]:
orders = orders.drop_duplicates(subset=['№ зкз.Покуп','КССС мат'], ignore_index=True)

## Улучшение таблицы orders

In [2235]:
orders['Заявлено Покупателем, тн'] = orders['Заявлено Покупателем, тн']/1000

orders['Не исполнено по продукту, тн'] = orders['Не исполнено по продукту, тн']/1000

orders['Отгружено, тн'] = orders['Отгружено, тн']/1000

In [2236]:
def change_name_of_city(city):
    if city == 'Волгоградский НПЗ':
        return 'ВНП'
    elif city == 'Пермнефтеоргсинтез ООО':
        return 'ПНОС'
    else:
        return city

In [2237]:
orders['Пункт отгрузки'] = orders.apply(lambda x: change_name_of_city(x['Пункт отгрузки']), axis=1)

## Читаем файлы operfas и группируем (ТОРЖОК)

In [2238]:
for i in os.listdir('operfas'):
    print(i)

A2_E21092022.txt
2209.txt
V_H2109.txt


In [2239]:
columns = ['data', 'Пункт отгрузки', 'КССС мат', 'balances', 'element']

In [2240]:
list_operfas = []
for i in os.listdir("operfas"):
    i = pd.read_csv('operfas/'+i, sep='\s+', encoding='CP1251', header = None)
    
    list_operfas.append(i)


for i in list_operfas:
    i.dropna(axis=1, inplace=True)
    i.dropna(axis=0, how='all', inplace=True)
    i.columns = columns
    i['balances'] = i['balances'].apply(lambda x: float(x.split()[0].replace(',', '.')))
    

    
req_3_factory = pd.DataFrame(pd.concat(list_operfas, ignore_index=True))


In [2241]:
req_3_factory.drop(['data', 'element'], axis=1, inplace=True)

In [2242]:
fiz_ostatki = req_3_factory.groupby(['Пункт отгрузки','КССС мат'], as_index=False).sum()


In [2243]:
fiz_ostatki

,Пункт отгрузки,КССС мат,balances
0,1078,1968,18130.00
1,1078,1972,15540.00
2,1078,1990,1800.00
3,1078,2253,0.00
4,1078,2260,7920.00
...,...,...,...
1364,198349,3457584,82790.00
1365,198349,3460018,720.00
1366,198349,3461040,700.00
1367,198349,3464374,100325.36


обнуляем отрицательные значения

In [2244]:
#fiz_ostatki.loc[fiz_ostatki['balances'] < 0, 'balances'] = 0

In [2245]:
#fiz_ostatki[fiz_ostatki['balances']<0]['balances']

### Читаем файл "брак" и удаляем объемы из fiz_ostatki

In [2246]:
scrap = pd.read_excel("spravochnik.xlsx", sheet_name='брак', usecols=[0,2,3],names=['КССС мат','balances', 'Пункт отгрузки'])
scrap = scrap.reindex(columns=['Пункт отгрузки','КССС мат', 'balances'])



In [2247]:
scrap

,Пункт отгрузки,КССС мат,balances
0,ПНОС,17367,5.632000
1,ПНОС,17437,16.500600
2,ПНОС,1561235,0.925000
3,ПНОС,3051150,0.017900
4,ПНОС,1561243,1.480000
...,...,...,...
152,ПНОС,1670489,0.320896
153,ПНОС,3099062,0.135792
154,ПНОС,3167334,0.430500
155,ПНОС,3167335,3.022194


In [2248]:
scrap['balances'] = scrap['balances']*1000
scrap['Пункт отгрузки'] = 198349

In [2249]:
temp_requirements = pd.merge(fiz_ostatki, scrap, on=['Пункт отгрузки','КССС мат'], how='left', suffixes=('_req', '_scrap'))

In [2250]:
temp_requirements['balances_scrap'].fillna(0, inplace=True)

In [2251]:
temp_requirements['balances'] = temp_requirements['balances_req']-temp_requirements['balances_scrap']

In [2252]:
temp_requirements

,Пункт отгрузки,КССС мат,balances_req,balances_scrap,balances
0,1078,1968,18130.00,0.0,18130.00
1,1078,1972,15540.00,0.0,15540.00
2,1078,1990,1800.00,0.0,1800.00
3,1078,2253,0.00,0.0,0.00
4,1078,2260,7920.00,0.0,7920.00
...,...,...,...,...,...
1364,198349,3457584,82790.00,0.0,82790.00
1365,198349,3460018,720.00,0.0,720.00
1366,198349,3461040,700.00,0.0,700.00
1367,198349,3464374,100325.36,0.0,100325.36


обнуляем отрицательные значения

In [2253]:
#temp_requirements.loc[temp_requirements['balances'] < 0, 'balances'] = 0

In [2254]:
temp_requirements[temp_requirements['balances']<0]

,Пункт отгрузки,КССС мат,balances_req,balances_scrap,balances
65,1078,225381,-4.455,0.000,-4.455000e+00
649,198349,17367,5068.800,5632.000,-5.632000e+02
799,198349,227272,445.440,626.400,-1.809600e+02
825,198349,1372222,61.632,359.520,-2.978880e+02
850,198349,1543274,1440.000,2880.000,-1.440000e+03
...,...,...,...,...,...
1136,198349,3149926,144.000,144.000,-2.842171e-14
1161,198349,3173877,599.808,1226.880,-6.270720e+02
1200,198349,3195944,26.880,26.880,-3.552714e-15
1225,198349,3233950,141.288,454.140,-3.128520e+02


In [2255]:
fiz_ostatki = temp_requirements.drop(['balances_scrap', 'balances_req'], axis=1)

In [2256]:
fiz_ostatki

,Пункт отгрузки,КССС мат,balances
0,1078,1968,18130.00
1,1078,1972,15540.00
2,1078,1990,1800.00
3,1078,2253,0.00
4,1078,2260,7920.00
...,...,...,...
1364,198349,3457584,82790.00
1365,198349,3460018,720.00
1366,198349,3461040,700.00
1367,198349,3464374,100325.36


In [2257]:
fiz_ostatki[fiz_ostatki['balances']<0]['balances']

65     -4.455000e+00
649    -5.632000e+02
799    -1.809600e+02
825    -2.978880e+02
850    -1.440000e+03
            ...     
1136   -2.842171e-14
1161   -6.270720e+02
1200   -3.552714e-15
1225   -3.128520e+02
1330   -2.371808e+03
Name: balances, Length: 18, dtype: float64

### меняем код города на название

In [2258]:
def area_change_name(area):
    if area == 1078:
        return 'ВНП'
    elif area == 198349:
        return 'ПНОС'
    elif area == 119388:
        return 'Тюменский Филиал'
    else:
        return area
        

In [2259]:
fiz_ostatki['Пункт отгрузки'] = fiz_ostatki.apply(lambda x: area_change_name(x['Пункт отгрузки']), axis=1)

In [2260]:
fiz_ostatki['balances'] = fiz_ostatki['balances']/1000

In [2261]:
fiz_ostatki[fiz_ostatki['balances']<0]['balances']

65     -4.455000e-03
649    -5.632000e-01
799    -1.809600e-01
825    -2.978880e-01
850    -1.440000e+00
            ...     
1136   -2.842171e-17
1161   -6.270720e-01
1200   -3.552714e-18
1225   -3.128520e-01
1330   -2.371808e+00
Name: balances, Length: 18, dtype: float64

## Расчет физических остатков

In [2262]:
fiz_ostatki = pd.merge(fiz_ostatki, orders[['Пункт отгрузки', 'КССС мат', 'Отгружено, тн']], how='left')
fiz_ostatki['Отгружено, тн'].fillna(0, inplace=True)
fiz_ostatki['cum_sum'] = fiz_ostatki.groupby(['Пункт отгрузки', 'КССС мат'])['Отгружено, тн'].cumsum()
fiz_ostatki = fiz_ostatki.drop_duplicates(subset=['Пункт отгрузки','КССС мат'], keep='last', ignore_index=True)


In [2263]:
fiz_ostatki[fiz_ostatki['balances']<0]['balances']

65     -4.455000e-03
649    -5.632000e-01
799    -1.809600e-01
825    -2.978880e-01
850    -1.440000e+00
            ...     
1136   -2.842171e-17
1161   -6.270720e-01
1200   -3.552714e-18
1225   -3.128520e-01
1330   -2.371808e+00
Name: balances, Length: 18, dtype: float64

In [2264]:
fiz_ostatki

,Пункт отгрузки,КССС мат,balances,"Отгружено, тн",cum_sum
0,ВНП,1968,18.13000,0.0,4.070
1,ВНП,1972,15.54000,0.0,6.105
2,ВНП,1990,1.80000,0.0,0.000
3,ВНП,2253,0.00000,0.0,0.000
4,ВНП,2260,7.92000,0.0,3.600
...,...,...,...,...,...
1364,ПНОС,3457584,82.79000,6.8,81.600
1365,ПНОС,3460018,0.72000,0.0,0.000
1366,ПНОС,3461040,0.70000,0.0,0.000
1367,ПНОС,3464374,100.32536,0.0,0.000


In [2265]:
fiz_ostatki['balances'] = (fiz_ostatki['balances'] - fiz_ostatki['cum_sum']).clip(lower=0)
fiz_ostatki.drop(['Отгружено, тн', 'cum_sum'], axis=1, inplace=True)
fiz_ostatki


,Пункт отгрузки,КССС мат,balances
0,ВНП,1968,14.06000
1,ВНП,1972,9.43500
2,ВНП,1990,1.80000
3,ВНП,2253,0.00000
4,ВНП,2260,4.32000
...,...,...,...
1364,ПНОС,3457584,1.19000
1365,ПНОС,3460018,0.72000
1366,ПНОС,3461040,0.70000
1367,ПНОС,3464374,100.32536


In [2266]:
fiz_ostatki[fiz_ostatki['balances']<0]['balances']

Series([], Name: balances, dtype: float64)

In [2267]:
# def raschet_fiz_ostatki(code_of_product, area, otgruzheno):
#     if otgruzheno!=0:
#         if fiz_ostatki[(fiz_ostatki['КССС мат']==code_of_product) & (fiz_ostatki['Пункт отгрузки']==area)].empty:
#             return 
#         if otgruzheno > fiz_ostatki.loc[(fiz_ostatki['КССС мат'] == code_of_product) & (fiz_ostatki['Пункт отгрузки']==area), 'balances'].iloc[0]:
    
#             fiz_ostatki.loc[(fiz_ostatki['КССС мат'] == code_of_product) & (fiz_ostatki['Пункт отгрузки']==area), 'balances'] = 0
#             return 
#         else: 
#             fiz_ostatki.loc[(fiz_ostatki['КССС мат']==code_of_product) & (fiz_ostatki['Пункт отгрузки']==area), 'balances'] -= otgruzheno
#             return 
    

In [2268]:
# orders.apply(lambda x: raschet_fiz_ostatki(x['КССС мат'], x['Пункт отгрузки'], x['Отгружено, тн']), axis=1)
# None

### Торжок

In [2269]:
torzhok = pd.read_excel('operfas_торжок/стоки_торжок.xlsx')

torzhok.drop(columns='Наименование', axis=1, inplace=True)

torzhok.rename(columns={'Кссс': 'КССС мат', 'Кг': 'balances'}, inplace=True)

torzhok = torzhok.reindex(columns=['Пункт отгрузки','КССС мат','balances'])

In [2270]:
torzhok.groupby(['Пункт отгрузки', 'КССС мат'], as_index=False).sum()

,Пункт отгрузки,КССС мат,balances
0,"ООО ""Тебойл Рус"" Москва",0.0,953.58
1,"ООО ""Тебойл Рус"" Москва",13105.0,0.00
2,"ООО ""Тебойл Рус"" Москва",17414.0,0.00
3,"ООО ""Тебойл Рус"" Москва",17747.0,0.00
4,"ООО ""Тебойл Рус"" Москва",17797.0,0.00
...,...,...,...
1864,"ООО ""Тебойл Рус"" Торжок",550047204.0,4475.52
1865,"ООО ""Тебойл Рус"" Торжок",550047222.0,3409.92
1866,"ООО ""Тебойл Рус"" Торжок",550050004.0,118.56
1867,"ООО ""Тебойл Рус"" Торжок",550058262.0,0.00


In [2271]:
torzhok['balances'] = torzhok['balances']/1000

In [2272]:
fiz_ostatki = pd.concat([fiz_ostatki, torzhok], ignore_index=True)

Заменим отрицательные значения на ноль

In [2273]:
fiz_ostatki.loc[fiz_ostatki['balances'] < 0, 'balances'] = 0

In [2274]:
fiz_ostatki

,Пункт отгрузки,КССС мат,balances
0,ВНП,1968.0,14.060000
1,ВНП,1972.0,9.435000
2,ВНП,1990.0,1.800000
3,ВНП,2253.0,0.000000
4,ВНП,2260.0,4.320000
...,...,...,...
3273,"ООО ""Тебойл Рус"" С.-Петербург",187548.0,0.000000
3274,"ООО ""Тебойл Рус"" С.-Петербург",187567.0,0.000000
3275,"ООО ""Тебойл Рус"" С.-Петербург",187682.0,0.000000
3276,"ООО ""Тебойл Рус"" С.-Петербург",3453873.0,0.000000


In [2275]:
orders

,№ зкз.Покуп,Дата разнарядки с учетом приоритета,КССС мат,Номенклатура материала,"Заявлено Покупателем, тн","Не исполнено по продукту, тн","Отгружено, тн",Покупатель,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,Инкотермс,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Рынок,Пункт отгрузки,Предписания по таре,Период,Инкотермс 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта,Стоимость,Валюта
0,5486287,2022-09-05,1599892,"Масло L GENESIS SPEC A5/В5 5W30;б.216,5л",7.480000,0.000000,7.48,"АО ""Авилон АГ""",Авто для фасовки,C,NaN,11:11:29,2022-09-08,08:59:01,2022-09-30,2022-09-07,DDP,Оптовые поставки,20L0408,2022-09-05,2022-09-05,Внутренний рынок,ПНОС,крупная фасовка,202209,Москва,77.0,г. Москва,Автотранспорт(Пермь->Москва),"МОСКОВСКАЯ ОБЛАСТЬ, Г. КОТЕЛЬНИКИ, КОММЕРЧЕСК...",NaN,7480.0,Фирм.не аналог имп/з,"АО ""Авилон АГ""",192281,Автотранспорт,A,NaN,NaN
1,5486138,2022-09-01,227349,"Масло Л АВАНГАРД УЛЬТРА,10W30; б.216,5л",0.900000,0.000000,0.90,"ООО ""ЛУКОЙЛ-Пермнефтеоргсинтез""",Авто для фасовки,NaN,"Недогруз Август 2022 г., разнярядка № 0005485...",00:00:00,1899-12-30,13:45:31,2022-09-30,2022-09-04,DDU,внутрикорп. Транзит,13L0555,2022-09-02,2022-09-06,Внутренний рынок,ПНОС,крупная фасовка,202209,г. Тюмень,77.0,г. Москва,Автотранспорт(Пермь->Тюмень),"П. БОГАНДИНСКИЙ, УЛ. НЕФТЯННИКОВ, 9",NaN,900.0,Фирм.не аналог имп/з,"Филиал ООО ""ЛЛК-Интернешнл"" в Тюмени",198349,Автотранспорт,A,NaN,NaN
2,5486125,2022-09-02,3435821,Масло Л АВ ПРОФ М5 10W40 б.175кг,0.000000,0.000000,0.00,"ООО ""Виланд""",Авто для фасовки,B,NaN,12:54:05,2022-09-06,12:38:04,2022-09-30,2022-09-04,CPT,Белоруссия,19L0442,2022-09-02,2022-09-13,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,0.0,Нет данных,Автотранспорт(Пермь->г. Минск),"ТЖ:МИНСКАЯ ОБЛ.,ЩОМЫСЛИЦКИЙ С/С,17-Й КМ. А/Д ...",NaN,0.0,NaN,"ООО ""Виланд""",184884,Автотранспорт,E,NaN,NaN
3,5492131,2022-09-07,189509,"Масло Л ЛЮКС ТУРБО ДИЗЕЛЬ 10W40 б.216,5л",0.720000,0.000000,0.72,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07,11:06:27,2022-09-30,2022-09-09,CPT,Белоруссия,19L0014,2022-09-07,2022-09-07,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,7.0,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,720.0,Фирм. аналог имп/зам,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN
4,5492131,2022-09-07,14913,"Масло Л СУПЕР, 10W40, SG/CD; б.216,5л",5.760000,0.000000,5.76,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07,11:06:27,2022-09-30,2022-09-09,CPT,Белоруссия,19L0014,2022-09-07,2022-09-07,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,7.0,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,5760.0,Фирм.не аналог имп/з,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21346,5494589,2022-09-24,19545,Масло Л ТРАНС ТМ-5 75W90 нк.4л,1.278720,1.278720,0.00,"ООО ""Теплоград""",Трансконтейнер,G,"#Автодоставка. Грузопотребитель: ООО ""Теплогр...",11:12:11,2022-09-22,11:12:05,2022-09-30,2022-09-24,DDP,Оптовые поставки,20L0196,2022-09-22,2022-09-22,Внутренний рынок,ПНОС,средняя и мелкая фc.,202209,ст. Батарейная,38.0,Иркутская обл.,Пермь->Блочная->Батарейная,"Г. ИРКУТСК, УЛ.ШИРЯМОВА 2, ТЕЛ: (3952)798-809",B2C,0.0,Фирм.не аналог имп/з,"ООО ""Теплоград""",53446,Мультимодальная,G,"757498,36",RUB
21347,5494589,2022-09-24,19543,"Масло Л ТРАНС ТМ-5, 75W90; нк.1л",4.699296,4.699296,0.00,"ООО ""Теплоград""",Трансконтейнер,G,"#Автодоставка. Грузопотребитель: ООО ""Теплогр...",11:12:11,2022-09-22,11:12:05,2022-09-30,2022-09-24,DDP,Оптовые поставки,20L0196,2022-09-22,2022-09-22,Внутренний рынок,ПНОС,средняя и мелкая фc.,202209,ст. Батарейная,38.0,Иркутская 

## Распределение остатков

In [2276]:
orders = pd.merge(orders, fiz_ostatki, how="left")
orders.balances.fillna(0, inplace=True)
orders["requires_cum"] = orders.groupby(["Пункт отгрузки", "КССС мат"])['Не исполнено по продукту, тн'].cumsum()
orders["Готово к отгрузке по продукту, тн"] = orders['Не исполнено по продукту, тн'].clip(upper=orders['balances'] - orders['requires_cum'] + orders['Не исполнено по продукту, тн']).clip(lower=0)

### Присвоение даты

In [2277]:
graphic = pd.read_excel('spravochnik.xlsx', sheet_name='график')

In [2278]:
graphic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Пункт отгрузки          122 non-null    object        
 1   Тип фасовки             122 non-null    object        
 2   Код КССС                122 non-null    int64         
 3   Наименование продукции  122 non-null    object        
 4   Дата_производства       122 non-null    datetime64[ns]
 5   Объем_фасовки, т        122 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 5.8+ KB


In [2279]:
graphic.head()

,Пункт отгрузки,Тип фасовки,Код КССС,Наименование продукции,Дата_производства,"Объем_фасовки, т"
0,ПНОС,1,3148620,Масло L. GENESIS UNIVERSAL 5W30; нк.1л.,2022-07-30,6.0
1,ПНОС,1,3148630,Масло L. GENESIS UNIVERSAL 5W40; нк.1л.,2022-07-29,29.0
2,ПНОС,1,19441,"Масло Л. СУПЕР 5W40, SG/CD; нк.1л.",2022-07-27,25.0
3,ПНОС,1,19191,"Масло Л. СУПЕР 10W40, SG/CD; нк.1л.",2022-07-28,35.0
4,ПНОС,1,19191,"Масло Л. СУПЕР 10W40, SG/CD; нк.1л.",2022-07-29,10.0


In [2280]:
graphic['Дата_производства'] = graphic['Дата_производства']+pd.Timedelta(days=2)

In [2281]:
graphic[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==3148620)]['Дата_производства'].dt.date.iloc[0]

datetime.date(2022, 8, 1)

### Добавляем колонку того, сколько осталось произвести

In [2282]:
orders['осталось произвести, тн'] = (orders['Не исполнено по продукту, тн'] - orders['Готово к отгрузке по продукту, тн']).clip(lower=0)

In [2283]:
orders[orders['осталось произвести, тн']<0]['осталось произвести, тн']

Series([], Name: осталось произвести, тн, dtype: float64)

## Распределение даты

In [2284]:
def data_shipment(city, code, required_to_make):
    # проверяем есть ли вообще в списке graphic
    if graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)].empty:
        return np.nan
    # если в таблице orders не требуется, то мы ставим черточку
    elif required_to_make == 0:
        return np.nan
    # иначе если есть и объема в графике достаточно, то мы    
    elif graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[0] >= required_to_make:        
        # вычитаем из объема
        # добавить определенное дату, так как будет удалять все значения где код и пункт совпадаем
        data_of_making = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                                    (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
                                    & (graphic['Код КССС']==code), 'Дата_производства'].iloc[0]),'Дата_производства'].iloc[0]
        graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & (graphic['Дата_производства']==data_of_making),
                    'Объем_фасовки, т'] -= required_to_make
        # и возвращаем дату
        data = graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]['Дата_производства'].dt.date.iloc[0]
        return data
    # иначе если недостаточно объема чем требуется ...
    else:
        # иначе если возвращается только одна дата, возвращает -
        #graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'] = 0
        if len(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]) == 1:
            return np.nan
        # и возращается несколько дат
        elif len(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]) > 1:
            # вводим переменную длины, чтобы не выйти за пределы
            len_of_dates = len(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)])
            # вводим счетчик для того чтобы присвоить правильную дату
            i = 0
            # создаем накопительную переменную 
            storage_var = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[0]
            list_of_dates_to_be_deleted = []

            # пока объем фасовки меньше, чем требуется
            while storage_var<required_to_make and len_of_dates>1:
                len_of_dates-=1
                # переходим к следующей
                i+=1
                # и прибавляем к накопительной переменной
                storage_var+= graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[i]
                list_of_dates_to_be_deleted.append(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                                             (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
                                            & (graphic['Код КССС']==code), 'Дата_производства'].iloc[i-1]),
                                             'Дата_производства'].iloc[0])                    
            #если требуемый объем не накопился то возвращает -
            if storage_var<required_to_make:
                return np.nan
            # иначе
            else:
                #нужно обнулить или удалить предыдущие даты, которые схлопнулись
                for j in (list_of_dates_to_be_deleted):
                    graphic.drop(graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                           (graphic['Дата_производства']==j)].index, inplace=True)
                    i-=1
                data_of_making = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                                             (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
                                            & (graphic['Код КССС']==code), 'Дата_производства'].iloc[i]),
                                             'Дата_производства'].iloc[0]
        
            # вычитаем
            graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                        (graphic['Дата_производства']==data_of_making), 'Объем_фасовки, т'] = storage_var-required_to_make
 
            data = graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]['Дата_производства'].dt.date.iloc[i] 
            return data

In [2285]:

orders['дата производства'] = orders.apply(lambda x: data_shipment(x['Пункт отгрузки'], x['КССС мат'], x['осталось произвести, тн']), axis=1)

In [2286]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21351 entries, 0 to 21350
Data columns (total 44 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   № зкз.Покуп                          21351 non-null  int64         
 1   Дата разнарядки с учетом приоритета  21351 non-null  datetime64[ns]
 2   КССС мат                             21351 non-null  int64         
 3   Номенклатура материала               21351 non-null  object        
 4   Заявлено Покупателем, тн             21351 non-null  float64       
 5   Не исполнено по продукту, тн         21351 non-null  float64       
 6   Отгружено, тн                        21351 non-null  float64       
 7   Покупатель                           21351 non-null  object        
 8   Вид транспортного средства           21351 non-null  object        
 9   СтКМ                                 17935 non-null  object        
 10  Комментари

In [2287]:
orders

,№ зкз.Покуп,Дата разнарядки с учетом приоритета,КССС мат,Номенклатура материала,"Заявлено Покупателем, тн","Не исполнено по продукту, тн","Отгружено, тн",Покупатель,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,Инкотермс,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Рынок,Пункт отгрузки,Предписания по таре,Период,Инкотермс 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта,Стоимость,Валюта,balances,requires_cum,"Готово к отгрузке по продукту, тн","осталось произвести, тн",дата производства
0,5486287,2022-09-05,1599892,"Масло L GENESIS SPEC A5/В5 5W30;б.216,5л",7.480000,0.000000,7.48,"АО ""Авилон АГ""",Авто для фасовки,C,NaN,11:11:29,2022-09-08,08:59:01,2022-09-30,2022-09-07,DDP,Оптовые поставки,20L0408,2022-09-05,2022-09-05,Внутренний рынок,ПНОС,крупная фасовка,202209,Москва,77.0,г. Москва,Автотранспорт(Пермь->Москва),"МОСКОВСКАЯ ОБЛАСТЬ, Г. КОТЕЛЬНИКИ, КОММЕРЧЕСК...",NaN,7480.0,Фирм.не аналог имп/з,"АО ""Авилон АГ""",192281,Автотранспорт,A,NaN,NaN,3.400000,0.000000,0.0,0.000000,NaN
1,5486138,2022-09-01,227349,"Масло Л АВАНГАРД УЛЬТРА,10W30; б.216,5л",0.900000,0.000000,0.90,"ООО ""ЛУКОЙЛ-Пермнефтеоргсинтез""",Авто для фасовки,NaN,"Недогруз Август 2022 г., разнярядка № 0005485...",00:00:00,1899-12-30,13:45:31,2022-09-30,2022-09-04,DDU,внутрикорп. Транзит,13L0555,2022-09-02,2022-09-06,Внутренний рынок,ПНОС,крупная фасовка,202209,г. Тюмень,77.0,г. Москва,Автотранспорт(Пермь->Тюмень),"П. БОГАНДИНСКИЙ, УЛ. НЕФТЯННИКОВ, 9",NaN,900.0,Фирм.не аналог имп/з,"Филиал ООО ""ЛЛК-Интернешнл"" в Тюмени",198349,Автотранспорт,A,NaN,NaN,2.880000,0.000000,0.0,0.000000,NaN
2,5486125,2022-09-02,3435821,Масло Л АВ ПРОФ М5 10W40 б.175кг,0.000000,0.000000,0.00,"ООО ""Виланд""",Авто для фасовки,B,NaN,12:54:05,2022-09-06,12:38:04,2022-09-30,2022-09-04,CPT,Белоруссия,19L0442,2022-09-02,2022-09-13,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,0.0,Нет данных,Автотранспорт(Пермь->г. Минск),"ТЖ:МИНСКАЯ ОБЛ.,ЩОМЫСЛИЦКИЙ С/С,17-Й КМ. А/Д ...",NaN,0.0,NaN,"ООО ""Виланд""",184884,Автотранспорт,E,NaN,NaN,4.200000,0.000000,0.0,0.000000,NaN
3,5492131,2022-09-07,189509,"Масло Л ЛЮКС ТУРБО ДИЗЕЛЬ 10W40 б.216,5л",0.720000,0.000000,0.72,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07,11:06:27,2022-09-30,2022-09-09,CPT,Белоруссия,19L0014,2022-09-07,2022-09-07,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,7.0,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,720.0,Фирм. аналог имп/зам,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN,0.000000,0.000000,0.0,0.000000,NaN
4,5492131,2022-09-07,14913,"Масло Л СУПЕР, 10W40, SG/CD; б.216,5л",5.760000,0.000000,5.76,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07,11:06:27,2022-09-30,2022-09-09,CPT,Белоруссия,19L0014,2022-09-07,2022-09-07,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,7.0,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,5760.0,Фирм.не аналог имп/з,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN,63.180000,0.000000,0.0,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21346,5494589,2022-09-24,19545,Масло Л ТРАНС ТМ-5 75W90 нк.4л,1.278720,1.278720,0.00,"ООО ""Теплоград""",Трансконтейнер,G,"#Автодоставка. Грузопотребитель: ООО ""Теплогр...",11:12:11,2022-09-22,11:12:05,2022-09-30,2022-09-24,DDP,Оптовые поставки,20L0196,2022-09-22,2022-09-22,Внутренний рынок,ПНОС,средняя и мелкая фc.,202209,ст. Батарейная,38.0,Иркутская обл.,Пермь->Блочная->Батарейная,"Г. ИРКУТСК, УЛ.ШИРЯМОВА 2, ТЕЛ: (3952)798-809",B2C,0.0,Фирм.не аналог имп/з,"ООО ""Теплоград""",53446,Мультимодальная,G,"757498,36",RUB,0.412032,179.319168,0.0,1.278720,NaN
21347,5494589,2022-09-24,

In [2288]:
orders.iloc[2603]

№ зкз.Покуп                                                            5489968
Дата разнарядки с учетом приоритета                        2022-04-22 00:00:00
КССС мат                                                                227349
Номенклатура материала                 Масло Л АВАНГАРД УЛЬТРА,10W30; б.216,5л
Заявлено Покупателем, тн                                                   3.6
                                                        ...                   
balances                                                                  2.88
requires_cum                                                             25.38
Готово к отгрузке по продукту, тн                                          0.0
осталось произвести, тн                                                    3.6
дата производства                                                   2022-07-29
Name: 2603, Length: 44, dtype: object

In [2289]:
orders.loc[2603,'дата производства']

datetime.date(2022, 7, 29)

In [2290]:
orders['дата производства'] = pd.to_datetime(orders['дата производства'])

In [2291]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21351 entries, 0 to 21350
Data columns (total 44 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   № зкз.Покуп                          21351 non-null  int64         
 1   Дата разнарядки с учетом приоритета  21351 non-null  datetime64[ns]
 2   КССС мат                             21351 non-null  int64         
 3   Номенклатура материала               21351 non-null  object        
 4   Заявлено Покупателем, тн             21351 non-null  float64       
 5   Не исполнено по продукту, тн         21351 non-null  float64       
 6   Отгружено, тн                        21351 non-null  float64       
 7   Покупатель                           21351 non-null  object        
 8   Вид транспортного средства           21351 non-null  object        
 9   СтКМ                                 17935 non-null  object        
 10  Комментари

### Признак перемещение

In [2292]:
movement_feature = pd.read_excel('spravochnik.xlsx', sheet_name='признак перемещение', usecols=[0,1], names=['Пункт отгрузки','КССС мат'])


In [2293]:
movement_feature.drop_duplicates(inplace=True)

In [2294]:
movement_feature[movement_feature.duplicated()]

,Пункт отгрузки,КССС мат


In [2295]:
orders = pd.merge(orders, movement_feature, on=['Пункт отгрузки', 'КССС мат'], how='left', indicator=True)

In [2296]:
orders

,№ зкз.Покуп,Дата разнарядки с учетом приоритета,КССС мат,Номенклатура материала,"Заявлено Покупателем, тн","Не исполнено по продукту, тн","Отгружено, тн",Покупатель,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,Инкотермс,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Рынок,Пункт отгрузки,Предписания по таре,Период,Инкотермс 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта,Стоимость,Валюта,balances,requires_cum,"Готово к отгрузке по продукту, тн","осталось произвести, тн",дата производства,_merge
0,5486287,2022-09-05,1599892,"Масло L GENESIS SPEC A5/В5 5W30;б.216,5л",7.480000,0.000000,7.48,"АО ""Авилон АГ""",Авто для фасовки,C,NaN,11:11:29,2022-09-08,08:59:01,2022-09-30,2022-09-07,DDP,Оптовые поставки,20L0408,2022-09-05,2022-09-05,Внутренний рынок,ПНОС,крупная фасовка,202209,Москва,77.0,г. Москва,Автотранспорт(Пермь->Москва),"МОСКОВСКАЯ ОБЛАСТЬ, Г. КОТЕЛЬНИКИ, КОММЕРЧЕСК...",NaN,7480.0,Фирм.не аналог имп/з,"АО ""Авилон АГ""",192281,Автотранспорт,A,NaN,NaN,3.400000,0.000000,0.0,0.000000,NaT,both
1,5486138,2022-09-01,227349,"Масло Л АВАНГАРД УЛЬТРА,10W30; б.216,5л",0.900000,0.000000,0.90,"ООО ""ЛУКОЙЛ-Пермнефтеоргсинтез""",Авто для фасовки,NaN,"Недогруз Август 2022 г., разнярядка № 0005485...",00:00:00,1899-12-30,13:45:31,2022-09-30,2022-09-04,DDU,внутрикорп. Транзит,13L0555,2022-09-02,2022-09-06,Внутренний рынок,ПНОС,крупная фасовка,202209,г. Тюмень,77.0,г. Москва,Автотранспорт(Пермь->Тюмень),"П. БОГАНДИНСКИЙ, УЛ. НЕФТЯННИКОВ, 9",NaN,900.0,Фирм.не аналог имп/з,"Филиал ООО ""ЛЛК-Интернешнл"" в Тюмени",198349,Автотранспорт,A,NaN,NaN,2.880000,0.000000,0.0,0.000000,NaT,both
2,5486125,2022-09-02,3435821,Масло Л АВ ПРОФ М5 10W40 б.175кг,0.000000,0.000000,0.00,"ООО ""Виланд""",Авто для фасовки,B,NaN,12:54:05,2022-09-06,12:38:04,2022-09-30,2022-09-04,CPT,Белоруссия,19L0442,2022-09-02,2022-09-13,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,0.0,Нет данных,Автотранспорт(Пермь->г. Минск),"ТЖ:МИНСКАЯ ОБЛ.,ЩОМЫСЛИЦКИЙ С/С,17-Й КМ. А/Д ...",NaN,0.0,NaN,"ООО ""Виланд""",184884,Автотранспорт,E,NaN,NaN,4.200000,0.000000,0.0,0.000000,NaT,left_only
3,5492131,2022-09-07,189509,"Масло Л ЛЮКС ТУРБО ДИЗЕЛЬ 10W40 б.216,5л",0.720000,0.000000,0.72,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07,11:06:27,2022-09-30,2022-09-09,CPT,Белоруссия,19L0014,2022-09-07,2022-09-07,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,7.0,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,720.0,Фирм. аналог имп/зам,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN,0.000000,0.000000,0.0,0.000000,NaT,left_only
4,5492131,2022-09-07,14913,"Масло Л СУПЕР, 10W40, SG/CD; б.216,5л",5.760000,0.000000,5.76,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07,11:06:27,2022-09-30,2022-09-09,CPT,Белоруссия,19L0014,2022-09-07,2022-09-07,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,7.0,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,5760.0,Фирм.не аналог имп/з,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN,63.180000,0.000000,0.0,0.000000,NaT,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21346,5494589,2022-09-24,19545,Масло Л ТРАНС ТМ-5 75W90 нк.4л,1.278720,1.278720,0.00,"ООО ""Теплоград""",Трансконтейнер,G,"#Автодоставка. Грузопотребитель: ООО ""Теплогр...",11:12:11,2022-09-22,11:12:05,2022-09-30,2022-09-24,DDP,Оптовые поставки,20L0196,2022-09-22,2022-09-22,Внутренний рынок,ПНОС,средняя и мелкая фc.,202209,ст. Батарейная,38.0,Иркутская обл.,Пермь->Блочная->Батарейная,"Г. ИРКУТСК, УЛ.ШИРЯМОВА 2, ТЕЛ: (3952)798-809",B2C,0.0,Фирм.не аналог имп/з,"ООО ""Теплоград""",53446,Мультимодальная,G,"757498,36",RUB,0.412032,17

###  Добавление условного столбца

In [2297]:
orders['Готово к отгрузке по продукту, тн'].fillna(0, inplace=True)

In [2298]:
orders['Готово к отгрузке по продукту, тн'] = orders['Готово к отгрузке по продукту, тн'].round(5)
orders['Не исполнено по продукту, тн'] = orders['Не исполнено по продукту, тн'].round(5)

In [2299]:
orders['Готовность продукта КС'] = 'Частично произведено'

In [2300]:
orders.loc[orders['дата производства'].notna(), 'Готовность продукта КС'] = 'В графике'
orders.loc[orders['Не исполнено по продукту, тн'] <= orders['Готово к отгрузке по продукту, тн'], 
           'Готовность продукта КС'] = 'Произведено'
orders.loc[(orders['Отгружено, тн'] == 0)&(orders['Готово к отгрузке по продукту, тн']==0) 
           &(orders['Готовность продукта КС'] != 'В графике'),'Готовность продукта КС'] = 'В обработке'
orders.loc[orders['Не исполнено по продукту, тн'] == 0, 'Готовность продукта КС'] = 'Отгружено'
orders.loc[orders['Заявлено Покупателем, тн'] == 0, 'Готовность продукта КС'] = 'Удалено'

orders.loc[(orders['осталось произвести, тн'] > 0) & (orders['_merge'] == 'both'), 
           'Готовность продукта КС'] = 'К перемещению'


In [2301]:
orders.loc[orders['дата производства'].notna()]

,№ зкз.Покуп,Дата разнарядки с учетом приоритета,КССС мат,Номенклатура материала,"Заявлено Покупателем, тн","Не исполнено по продукту, тн","Отгружено, тн",Покупатель,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,Инкотермс,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Рынок,Пункт отгрузки,Предписания по таре,Период,Инкотермс 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта,Стоимость,Валюта,balances,requires_cum,"Готово к отгрузке по продукту, тн","осталось произвести, тн",дата производства,_merge,Готовность продукта КС
514,5487045,2022-06-14,11466,Масло МГЕ-10А; бид.18л,0.04500,0.04500,0.0,"ООО ""ЛУКОЙЛ-Коми""",Трансконтейнер,A,"Недогруз Август 2022 г., разнярядка № 0005476...",08:59:03,2022-06-20,10:24:26,2022-09-30,2022-09-07,DDU,Оптовые поставки,11L0542,2022-09-05,2022-09-05,Внутренний рынок,ВНП,средняя и мелкая фc.,202209,ст. Усинск,11.0,Республика Коми,Волгоград->Сарепта->Усинск,СТ. УСИНСК,NaN,0.0,Нефирменные масла,"ООО ""СГК""",4925,Мультимодальная,A,NaN,NaN,3.285000,3.28500,0.04500,4.440892e-16,2022-08-04,both,К перемещению
2603,5489968,2022-04-22,227349,"Масло Л АВАНГАРД УЛЬТРА,10W30; б.216,5л",3.60000,3.60000,0.0,"ООО ""Торговый Дом ""КАРАГИ-ПЛЮС""",Трансконтейнер,F,"Недогруз Август 2022 г., разнярядка №5478964 ...",10:05:27,2022-06-09,12:50:05,2022-09-30,2022-07-09,DDP,внутрикорп. Транзит,20L0210,2022-05-09,2022-05-09,Внутренний рынок,ПНОС,крупная фасовка,202209,ст. Нижний Бестях,50.0,Московская обл.,Пермь->Блочная->Нижний Бестях,"Г. ЯКУТСК, УЛ. ЧЕЛЮСКИНА, Д. 37/6",B2B,0.0,Фирм.не аналог имп/з,"ООО ""ТК Якутский Транзит""",2011805,Мультимодальная,G,"1033353,72",RUB,2.880000,25.38000,0.00000,3.600000e+00,2022-07-29,both,К перемещению
3571,5487935,2022-05-13,196256,"Масло ЛУКОЙЛ ЛЮКС синт,5W30, SL/CF;нк.4л",0.61344,0.61344,0.0,"ООО ТК ""ПромАвтоСнаб""",Авто для фасовки,F,"Недогруз Август 2022 г., разнярядка № 0005478...",09:55:03,2022-09-21,10:43:11,2022-09-30,2022-07-09,CPT,Оптовые поставки,21L0071,2022-05-09,2022-05-09,Внутренний рынок,ПНОС,средняя и мелкая фc.,202209,Санкт-Петербург,78.0,г. Санкт-Петербург,Автотранспорт(Пермь->Архангельск),"Г. АРХАНГЕЛЬСК, УЛ. ДАЧНАЯ, 70",B2B+B2C,0.0,Фирм. аналог имп/зам,"ООО ТК ""ПромАвтоСнаб""",140424,Автотранспорт,G,"343063,51",RUB,3.244416,3.68064,0.17722,4.362240e-01,2022-07-29,both,К перемещению
3650,5488123,2022-05-13,196256,"Масло ЛУКОЙЛ ЛЮКС синт,5W30, SL/CF;нк.4л",3.68064,3.68064,0.0,"ООО ТК ""ПромАвтоСнаб""",Авто для фасовки,D,"Недогруз Август 2022 г., разнярядка № 0005480...",15:53:37,2022-09-19,10:46:55,2022-09-30,2022-07-09,CPT,Оптовые поставки,21L0071,2022-05-09,2022-05-09,Внутренний рынок,ПНОС,средняя и мелкая фc.,202209,Санкт-Петербург,78.0,г. Санкт-Петербург,Автотранспорт(Пермь->Санкт-Петербург),"ЛЕНИНГРАДСКАЯ ОБЛАСТЬ, ЛОМОНОСОВСКИЙ Р-Н, КРА...",B2B+B2C,0.0,Фирм. аналог имп/зам,"ООО ТК ""ПромАвтоСнаб""",140424,Автотранспорт,G,"2058381,06",RUB,3.244416,7.36128,0.00000,3.680640e+00,2022-07-29,both,К перемещению
3736,5488938,2022-05-13,196256,"Масло ЛУКОЙЛ ЛЮКС синт,5W30, SL/CF;нк.4л",1.84032,1.84032,0.0,"ООО ""РеалНефтеПродукт""",Трансконтейнер,D,"Недогруз Август 2022 г., разнярядка № 0005479...",14:52:34,2022-08-24,11:15:23,2022-09-30,2022-07-09,CPT,Оптовые поставки,20L0200,2022-05-09,2022-05-09,Внутренний рынок,ПНОС,средняя и мелкая фc.,202209,ст. Базаиха,24.0,Красноярский край,Пермь->Блочная->Базаиха,СТ. БАЗАИХА,B2B+B2C,0.0,Фирм. аналог имп/зам,"ООО ""РеалНефтеПродукт""",266555,Мультимодальная 2,G,"1029190,54",RUB,3.244416,9.20160,0.00000,1.840320e+00,2022-07-29,both,К перемещению
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21225,5488769,2022-09-24,18477,"Масло ЛУКОЙЛ ДИЗЕЛЬ М-10ДМ; б.216,5л",2.22000,2.22000,0.0,"ООО ""Авангард-Чита""",Трансконтейнер,B,"

### Просуммировать по разнарядке

в этом случае функция summ_by_order вызывается на объекте, к которой была применена функция и передается в аргументы

In [2302]:
 orders.groupby('№ зкз.Покуп', sort=False).sum()

,КССС мат,"Заявлено Покупателем, тн","Не исполнено по продукту, тн","Отгружено, тн",Период,Регион ГП,Исполнено факт.отгрузками,КССС Пок,balances,requires_cum,"Готово к отгрузке по продукту, тн","осталось произвести, тн"
№ зкз.Покуп,,,,,,,,,,,,
5486287,4636159,14.960000,0.00000,14.96000,404418,154.0,14960.00,384562,18.87000,0.000000,0.000,0.000000
5486138,227349,0.900000,0.00000,0.90000,202209,77.0,900.00,198349,2.88000,0.000000,0.000,0.000000
5486125,3435821,0.000000,0.00000,0.00000,202209,0.0,0.00,184884,4.20000,0.000000,0.000,0.000000
5492131,11025353,16.094880,0.00000,16.09488,1415463,49.0,16094.88,14777105,139.07748,0.000000,0.000,0.000000
5486990,3298747,0.975000,0.97500,0.00000,202209,16.0,0.00,818,4.09500,0.975000,0.975,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
5494556,6280925,0.000000,0.00000,0.00000,606627,114.0,0.00,160338,230.86000,698.700000,0.000,0.000000
5494557,78160,16.756800,16.75680,0.00000,808836,152.0,0.00,213784,0.65710,152.491200,0.000,16.756800
5494558,1882865,0.000000,0.00000,0.00000,808836,152.0,0.00,213784,0.91700,295.051200,0.000,0.000000


In [2303]:
def summ_by_order(group_by_column):
 
    summ = group_by_column['Готово к отгрузке по продукту, тн'].sum()
    group_by_column['Готово к отгрузке по разнарядке, тн'] = summ

    return group_by_column




In [2304]:
orders = orders.groupby('№ зкз.Покуп',sort=False).apply(summ_by_order)

### Заполним поле "ОБЪЕМ ПО РАЗНАРЯДКЕ"

In [2305]:
def summ_by_order_for_value(group):
    summ = group['Не исполнено по продукту, тн'].sum()
    group['ОБЪЕМ ПО РАЗНАРЯДКЕ (Не исполнено по разнарядке всего, тн)'] = summ

    return group

orders = orders.groupby('№ зкз.Покуп',sort=False).apply(summ_by_order_for_value)

In [2306]:
orders['% готовности разнарядки'] = orders['Готово к отгрузке по разнарядке, тн']/orders['ОБЪЕМ ПО РАЗНАРЯДКЕ (Не исполнено по разнарядке всего, тн)']
orders['% готовности разнарядки'] = orders['% готовности разнарядки'].round(3)
orders['% готовности разнарядки'].fillna(0.000, inplace=True)

## Признак диллера (справочник)

In [2307]:
diller = pd.read_excel('spravochnik.xlsx', usecols=['Contract_Number'], sheet_name='ДС')

In [2308]:
orders['Признак дилера'] = orders.apply(lambda x: 1 if x['Договор'] in diller['Contract_Number'].values else 0, axis=1)
#проверяем есть ли номер договора в справочнике, возрвращаем 1 если True, иначе 0d

In [2309]:
def ready_in_procent(row):
    if row==0:
        return 'Нулевая готовность(0%)'
    elif row<0.7:
        return 'Готовность ниже порога (<70%)'
    elif (row>=0.7) & (row<1):
        return 'Готовность выше порога(>=70% <100%)'
    else:
        return 'Полностью готова к отгрузке(100%)'
orders['Статус готовности по разнарядке'] = orders['% готовности разнарядки'].apply(ready_in_procent)

In [2310]:
orders['% готовности разнарядки']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
21346    0.0
21347    0.0
21348    0.0
21349    0.0
21350    0.0
Name: % готовности разнарядки, Length: 21351, dtype: float64

## Меняем колонки местами как в форме для рассылки

In [2311]:
orders = orders.reindex(columns=['Пункт отгрузки','Дата разнарядки с учетом приоритета','Дата закрытия разнарядки','№ зкз.Покуп','Договор',
                                 'Рынок', 'Инкотермс', 'Инкотермс 2', 'Вид транспортного средства', 'СтКМ продукта', 
                                 'КССС Пок', 'Покупатель',
                                 'КССС мат', 'Номенклатура материала', 'Готовность продукта', 'Готовность разнарядки', 'Готовность продукта КС',
                                 'Готовность разнарядки КС', 'Заявлено Покупателем, тн', 'Отгружено, тн',
                                 'Не исполнено по продукту, тн', 
                                 'ОБЪЕМ ПО РАЗНАРЯДКЕ (Не исполнено по разнарядке всего, тн)',
                                 'Готово к отгрузке по продукту, тн','Готово к отгрузке по разнарядке, тн',
                                 '% готовности разнарядки', 'Признак от 70%', 'Признак дилера', 
                                 'Признак большой разнарядки', 
                                 'Статус "on hold" в логистическом блоке (до отгрузки фур, загруженных на 100%)',
                                 'Фирм/ГОСТ', 'Пункт разгрузки', 'имя Грузополучателя', 'Комментарий',
                                 'Дата отчета', 'Условие отгрузки', 'Маршрут', 'объем одного поддона тн',
                                 'Не исполнено в поддонах шт', 'Готово к отгрузке поддонов шт',
                                  'ВремяСтКМ', 'ДатСтКМ', 'Время', 'ПланКОНдата', 
                                 'ПланНАЧдата', 'Район сбыта', 'ДатаСоздан', 'ДатаИзм',  'Предписания по таре',
                                 'Период', 'Регион ГП', 'Регион Грузополуч.',   'Тип дилера', 
                                 'Исполнено факт.отгрузками', 'Фирменная / не фирменная проду', 'Стоимость', 
                                 'Валюта', 'осталось произвести, тн', 'дата производства', 'Статус готовности по разнарядке'])

In [2312]:
orders.rename(columns = {'СтКМ продукта': 'СтКМ'}, inplace=True)

In [2313]:
orders['% готовности разнарядки']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
21346    0.0
21347    0.0
21348    0.0
21349    0.0
21350    0.0
Name: % готовности разнарядки, Length: 21351, dtype: float64

### заполним поле "готовность продукта"

In [2314]:
from datetime import datetime
current_date = datetime.now().date().strftime('%d.%m.%Y')
print(type(datetime.now().date()))
print(current_date)

<class 'datetime.date'>
01.10.2022


In [2315]:
def fill_column_product(readiness_of_the_KS_product, date_of_production):
    if pd.notnull(date_of_production):
        return date_of_production
    elif readiness_of_the_KS_product == 'Произведено':
        return current_date
    elif readiness_of_the_KS_product == 'Удалено':
        return '00.01.1900'
    elif readiness_of_the_KS_product == 'Отгружено':
        return '00.01.1900'
    elif readiness_of_the_KS_product == 'Частично произведено':
        return '-'
    elif readiness_of_the_KS_product == 'В обработке':
        return '-'
    

In [2316]:
orders['Готовность продукта'] = orders.apply(lambda x: fill_column_product(x['Готовность продукта КС'], x['дата производства']), axis=1)

### заполним поле "Готовность разнарядки КС"

In [2317]:
def fill_column_ks(group_by_column):
    
    if ('Удалено' == group_by_column['Готовность продукта КС']).all():
        group_by_column['Готовность разнарядки КС'] = 'Удалено'
        return group_by_column
    if ('Отгружено' == group_by_column['Готовность продукта КС']).all():
        group_by_column['Готовность разнарядки КС'] = 'Отгружено'
        return group_by_column
    if ('В обработке' == group_by_column['Готовность продукта КС']).any():
        group_by_column['Готовность разнарядки КС'] = 'В обработке'
        return group_by_column
    if ('Частично произведено' == group_by_column['Готовность продукта КС']).any():
        group_by_column['Готовность разнарядки КС'] = 'В обработке'
        return group_by_column
    if ('Произведено' == group_by_column['Готовность продукта КС']).any():
        group_by_column['Готовность разнарядки КС'] = 'Произведено'
        return group_by_column
    else:
        group_by_column['Готовность разнарядки КС'] = 'В обработке'
        return group_by_column
       



In [2318]:
orders = orders.groupby('№ зкз.Покуп',sort=False).apply(fill_column_ks)

### Заполним поле "Готовность разнарядки"

In [2319]:
def fill_column_order(readiness_of_the_KS_product):
    if readiness_of_the_KS_product == 'Произведено':
        return current_date
    elif readiness_of_the_KS_product == 'Удалено':
        return '-'
    elif readiness_of_the_KS_product == 'Отгружено':
        return '00.01.1900'
    elif readiness_of_the_KS_product == 'В обработке':
        return '-'

In [2320]:
orders['Готовность разнарядки'] = orders.apply(lambda x: fill_column_order(x['Готовность разнарядки КС']), axis=1)

### Заполним колонку "Признак от 70%"

In [2321]:
def fill_column_feature(feature):
    if feature >= 69.99:
        return 1
    else:
        return 0

In [2322]:
orders['% готовности разнарядки'] = orders['% готовности разнарядки'] * 100

In [2323]:
orders['Признак от 70%'] = orders.apply(lambda x: fill_column_feature(x['% готовности разнарядки']), axis=1)


In [2324]:
orders['Признак от 70%'].unique()

array([0, 1])

### Добавим колонку Группа ABC и нормативы

In [2325]:
normatives = pd.read_excel('spravochnik.xlsx', sheet_name='нормативы', 
                           usecols=[0,1,3,4],
                           names=['Пункт отгрузки', 'КССС мат', 
                                  'Группа ABC','Нормативный запас'] )



In [2326]:
orders = pd.merge(orders, normatives, on=['Пункт отгрузки','КССС мат'], how='left')

In [2327]:
orders['Группа ABC'] = orders['Группа ABC'].fillna('C')

In [2328]:
orders['Нормативный запас'] = orders['Нормативный запас'].fillna(0)

### Признак НПО и ИНТЕСМО

In [2329]:
npo = pd.read_excel('spravochnik.xlsx', sheet_name='признак НПО', names=['Договор', 'признак НПО']) 

In [2330]:
orders = pd.merge(orders, npo, on=['Договор'], how='left')

In [2331]:
intesmo = pd.read_excel('spravochnik.xlsx', sheet_name='признак Интесмо', usecols=[1,2]) 

In [2332]:
orders = pd.merge(orders, intesmo, on=['КССС мат'], how='left')

колонка с датой закрытия разнарядки

In [2333]:
orders['Дата закрытия разнарядки'] = orders['Дата разнарядки с учетом приоритета']

### добавим процент к колонке "Признак от 70%"

In [2334]:
orders['% готовности разнарядки'] = orders['% готовности разнарядки'].astype(str)+"%"

In [2335]:
orders['Дата отчета'] = current_date

### округление столбца Готово к отгрузке по продукту, тн


In [2336]:
orders['осталось произвести, тн'] = orders['осталось произвести, тн'].round(6)

# тест

### сохраним файл

In [2337]:
with pd.ExcelWriter('orders_output_4.xlsx', 
                    engine = 'xlsxwriter', 
                    datetime_format="DD.MM.YYYY") as writer:
    orders.to_excel(writer, sheet_name='Sheet1', startrow=1, header=False)

    # Get the xlsxwriter workbook and worksheet objects.
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']

    # Add a header format.
    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
#         'fg_color': '',
        'font_color': '#5D4B77',
        'border': 1
    })

    # Write the column headers with the defined format.
    for col_num, value in enumerate(orders.columns.values):
        worksheet.write(0, col_num + 1, value, header_format)
        
    (max_row, max_col) = orders.shape    
    worksheet.autofilter(0, 0, max_row, max_col)
    
    

# Конец файла